# Bayesian Optimization for Single-Interface Nanoparticle Discovery

**Notebook last update: 3/26/2021** (clean up)

This notebook contains the entire closed-loop process for SINP discovery with BO through SPBCL synthesis, STEM-EDS characterization, as reported in Wahl et al. *to be submitted* 2021.

In [1]:
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import os
import itertools
import io

from nanoparticle_project import EmbedCompGPUCB, get_comps, \
                        get_stoichiometric_formulas, compare_to_seed, load_np_data, update_with_new_data
from matminer.featurizers.composition import ElementProperty
# from pymatgen import Composition

from PCE_analysis import add_formula_col_OER, get_data, add_comp_col

path = os.getcwd()

 /home/dayong/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning:Failed to load image Python extension: /home/dayong/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZNK3c1010TensorImpl36is_contiguous_nondefault_policy_implENS_12MemoryFormatE


We will load our dataset into a pandas Dataframe and prepare for downstream modeling. We will prepare the feature space as described in the manuscript using the composition-derived descriptors of Ward et al.

## Prepare seed data and search space

In [2]:
import  torch
torch.cuda.is_available()
# torch.version.cuda

True

In [3]:
df = get_data('data/OER-Summary-LZ.xlsx')
df = add_formula_col_OER(df)
df = add_comp_col(df)


StrToComposition:   0%|          | 0/145 [00:00<?, ?it/s]

In [4]:
import re
# Define regular expression pattern to match element symbols
pattern = r'[A-Z][a-z]?'

# Create an empty set to store the unique elements
unique_elements = set()

# Loop through each chemical formula in the list
for formula in list(df['formula'].values):
    # Use regular expressions to find all element symbols in the formula
    elements = re.findall(pattern, formula)

    # Add the unique element symbols to the set
    unique_elements.update(elements)

# Convert the set to a list and sort it alphabetically
unique_elements = sorted(list(unique_elements))

print(unique_elements)
unique_elements = set(unique_elements)
len(unique_elements)

['Ag', 'Al', 'Ba', 'Bi', 'Ca', 'Cd', 'Ce', 'Co', 'Cr', 'Cu', 'Dy', 'Er', 'Eu', 'Fe', 'Ga', 'Gd', 'Hf', 'Ho', 'In', 'Ir', 'La', 'Li', 'Lu', 'Mg', 'Mn', 'Mo', 'Nb', 'Nd', 'Ni', 'Pb', 'Pd', 'Pr', 'Pt', 'Re', 'Rh', 'Ru', 'Sc', 'Sm', 'Sn', 'Ta', 'Tb', 'Ti', 'Tm', 'V', 'W', 'Y', 'Yb', 'Zn', 'Zr']


49

In [5]:
# _elts = ['Au%', 'Ag%', 'Cu%', 'Co%', 'Ni%', 'Pt%', 'Pd%', 'Sn%']
elem_necessay = set(['Ru'])             #bug1 to be recorded
elems_appd = set(unique_elements) - elem_necessay
elements = list(elems_appd)
elements.append('Ru')

df = df.sample(frac=1) # shuffling the dataframe
# df['target'] = -1*np.abs(df["Interfaces"]-1) # set target to single interface NPs
df = df[~df.duplicated()] # drop duplicates

# df['Composition'] = df.apply(get_comps,axis=1)
# df['n_elems'] = (df[unique_elements]>0).sum(axis=1)

ep = ElementProperty.from_preset(preset_name='magpie')
featurized_df = ep.featurize_dataframe(df[ ['composition'] ],'composition').drop('composition',axis=1)
featurized_df

ElementProperty:   0%|          | 0/145 [00:00<?, ?it/s]

,MagpieData minimum Number,MagpieData maximum Number,MagpieData range Number,MagpieData mean Number,MagpieData avg_dev Number,MagpieData mode Number,MagpieData minimum MendeleevNumber,MagpieData maximum MendeleevNumber,MagpieData range MendeleevNumber,MagpieData mean MendeleevNumber,...,MagpieData range GSmagmom,MagpieData mean GSmagmom,MagpieData avg_dev GSmagmom,MagpieData mode GSmagmom,MagpieData minimum SpaceGroupNumber,MagpieData maximum SpaceGroupNumber,MagpieData range SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber
34,28.0,44.0,16.0,39.2,6.72,44.0,56.0,61.0,5.0,57.5,...,0.595395,0.178618,0.250066,0.0,194.0,225.0,31.0,203.3,1.302000e+01,194.0
134,44.0,68.0,24.0,48.8,7.68,44.0,35.0,56.0,21.0,51.8,...,0.000000,0.000000,0.000000,0.0,194.0,194.0,0.0,194.0,2.842171e-14,194.0
2,3.0,44.0,41.0,35.8,13.12,44.0,1.0,56.0,55.0,45.0,...,0.000000,0.000000,0.000000,0.0,194.0,229.0,35.0,201.0,1.120000e+01,194.0
46,39.0,44.0,5.0,42.5,2.10,44.0,12.0,56.0,44.0,42.8,...,0.000000,0.000000,0.000000,0.0,194.0,194.0,0.0,194.0,2.842171e-14,194.0
132,44.0,67.0,23.0,46.3,4.14,44.0,33.0,56.0,23.0,53.7,...,0.000000,0.000000,0.000000,0.0,194.0,194.0,0.0,194.0,0.000000e+00,194.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,44.0,59.0,15.0,48.5,6.30,44.0,17.0,56.0,39.0,44.3,...,0.000000,0.000000,0.000000,0.0,194.0,194.0,0.0,194.0,2.842171e-14,194.0
85,44.0,74.0,30.0,53.0,12.60,44.0,51.0,56.0,5.0,54.5,...,0.000000,0.000000,0.000000,0.0,194.0,229.0,35.0,204.5,1.470000e+01,194.0
142,44.0,71.0,27.0,52.1,11.34,44.0,41.0,56.0,15.0,51.5,...,0.002247,0.000674,0.000944,0.0,194.0,194.0,0.0,194.0,2.842171e-14,194.0
31,27.0,44.0,17.0,38.9,7.14,44.0,56.0,58.0,2.0,56.6,...,1.548471,0.464541,0.650358,0.0,194.0,194.0,0.0,194.0,2.842171e-14,194.0


We should now create our search space *D*. First, we generate the composition grid. Then we featurize it as before using compositional descriptors, to generate our `candidate_feats` to search over using BO. Next, we remove any composition from our search space that is closer to a data point in our experimental seed than 5% on any axis.

In [6]:
'''TEST MODUlE'''
import itertools
import  numpy as np
# Define a list of 48 elements
import itertools
import  numpy as np
# Define a list of 48 elements
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import os
import itertools
import io

from nanoparticle_project import EmbedCompGPUCB, get_comps, \
                        get_stoichiometric_formulas, compare_to_seed, load_np_data, update_with_new_data
from matminer.featurizers.composition import ElementProperty
# from pymatgen import Composition
from PCE_analysis import add_formula_col_OER, get_data, add_comp_col


# get_stoichiometric_formulas(48)

In [7]:
len(elements)

49

In [10]:
D = get_stoichiometric_formulas(len(elements), set_ratios=[(4,6),(3,7),(1,9)], Ru_ratio=0.7)

candidate_data = pd.DataFrame.from_records(D, columns=elements)
# candidate_data['Pt%'] = 0.0
candidate_data[ candidate_data <0.00001 ] = 0.0
# candidate_data['composition'] = candidate_data.apply(get_comps, axis=1)
candidate_data['composition'] = candidate_data.apply(get_comps, axis=1)
candidate_feats = ep.featurize_dataframe(candidate_data, 'composition')
# candidate_feats = candidate_feats.drop(elements+['Pt%']+['Composition'],axis=1)   #drop some useless colums

'''我们从搜索空间中删除任何更接近实验种子中的数据点的成分，而不是任何轴上的 5%'''
# for ind,row in df[_elts].iterrows():
#     candidate_data = candidate_data[_elts][ np.any(np.abs(row - candidate_data)>=0.05,axis=1) ]
# candidate_feats = candidate_feats.loc[candidate_data.index]
candidate_feats.to_pickle('SearchSpace_3elems_Ru0.7.pkl')
candidate_feats.shape

ElementProperty:   0%|          | 0/6768 [00:00<?, ?it/s]

(6768, 182)

In [56]:
candidate_feats

,Ga,Dy,Bi,Yb,W,Zr,Rh,Pr,Gd,Cu,...,MagpieData range GSmagmom,MagpieData mean GSmagmom,MagpieData avg_dev GSmagmom,MagpieData mode GSmagmom,MagpieData minimum SpaceGroupNumber,MagpieData maximum SpaceGroupNumber,MagpieData range SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber
0,0.45,0.45,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,64.0,194.0,130.0,135.50,6.435000e+01,64.0
1,0.45,0.45,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,64.0,194.0,130.0,135.50,6.435000e+01,64.0
2,0.45,0.00,0.45,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,12.0,194.0,182.0,53.60,3.744000e+01,12.0
3,0.45,0.00,0.45,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,12.0,194.0,182.0,53.60,3.744000e+01,12.0
4,0.45,0.00,0.00,0.45,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,64.0,225.0,161.0,149.45,7.690500e+01,64.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6763,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,194.0,229.0,35.0,197.15,5.733000e+00,194.0
6764,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,194.0,194.0,0.0,194.00,2.842171e-14,194.0
6765,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,194.0,194.0,0.0,194.00,2.842171e-14,194.0
6766,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,194.0,229.0,35.0,197.15,5.733000e+00,194.0


# Closed-loop optimization procedure

We track the closed loop iterations below step-by-step, making suggestios and updating seed and candidate space with incoming data in each round. We follow this in unfolded form, so that we can closely inspect inputs and outputs in each round.

This is our initial data and the quaternary search space:

In [4]:
seed_df = df
seed_data = featurized_df
quaternaries = candidate_data[ ((candidate_data != 0).sum(axis=1) == 4)]
quaternary_feats = candidate_feats.loc[quaternaries.index]
round_number = 1

## Round 1
*Optimization agent's suggestions*:

In [5]:
agent = EmbedCompGPUCB(n_query=4)
suggestions = agent.get_hypotheses(candidate_data=quaternary_feats, seed_data=seed_data)
display(quaternaries.loc[ suggestions.index ])
compare_to_seed(quaternaries.loc[ suggestions.index ], seed_df)

- beta**0.5:0:  0.32187131685488807
- beta**0.5:1:  0.3219759073839426
- beta**0.5:2:  0.32207976468139243
- beta**0.5:3:  0.3221828987460505


,Au%,Ag%,Cu%,Co%,Ni%,Pt%,Pd%,Sn%
6291,0.3,0.0,0.2,0.1,0.4,0.0,0.0,0.0
6236,0.3,0.0,0.1,0.2,0.4,0.0,0.0,0.0
6922,0.4,0.0,0.1,0.1,0.4,0.0,0.0,0.0
6222,0.3,0.0,0.1,0.1,0.5,0.0,0.0,0.0


            Au%  Ag%   Cu%  Co%  Ni%  Pt%  Pd%  Sn%  target
suggested  0.30  0.0  0.20  0.1  0.4  0.0  0.0  0.0     NaN
inseed     0.23  0.0  0.17  0.0  0.6  0.0  0.0  0.0     0.0
            Au%  Ag%  Cu%   Co%   Ni%  Pt%  Pd%  Sn%  target
suggested  0.30  0.0  0.1  0.20  0.40  0.0  0.0  0.0     NaN
inseed     0.32  0.0  0.0  0.29  0.39  0.0  0.0  0.0     0.0
            Au%  Ag%  Cu%   Co%   Ni%  Pt%  Pd%   Sn%  target
suggested  0.40  0.0  0.1  0.10  0.40  0.0  0.0  0.00     NaN
inseed     0.37  0.0  0.0  0.18  0.36  0.0  0.0  0.09    -2.0
            Au%  Ag%   Cu%  Co%  Ni%  Pt%  Pd%  Sn%  target
suggested  0.30  0.0  0.10  0.1  0.5  0.0  0.0  0.0     NaN
inseed     0.23  0.0  0.17  0.0  0.6  0.0  0.0  0.0     0.0


*Experimental feedback in response to suggestions:*

In [6]:
new_raw_data = """
Co%	Ni%	Cu%	Au%
13.886	42.787	21.824	21.502
13.883	43.138	21.701	21.278
13.621	42.33	22.244	21.805
22.188	34.332	9.411	34.069
22.186	33.932	9.799	34.083
21.192	34.426	9.112	35.269
8.453	33.012	6.68	51.855
8.935	34.187	6.161	50.718
8.037	34.035	6.445	51.483
10.357	34.259	6.896	48.487
10.767	35.4	6.482	47.352
10.695	36.379	5.961	46.965
13.172	47.616	9.277	29.935
12.56	49.381	8.816	29.243
12.482	47.937	9.203	30.378
12.804	48.143	8.882	30.172
12.396	48.56	9.302	29.742
"""
seed_df, seed_data, quaternaries, quaternary_feats = update_with_new_data(suggestions, new_raw_data, seed_df, seed_data, 
                                                                 quaternaries, quaternary_feats, round_number=round_number,
                                                                 elements=elements, measured=0)
round_number+=1

ElementProperty:   0%|          | 0/16 [00:00<?, ?it/s]

## Round 2
*Optimization agent's suggestions*:

In [7]:
agent = EmbedCompGPUCB(n_query=4)
suggestions = agent.get_hypotheses(candidate_data=quaternary_feats, seed_data=seed_data)
display(quaternaries.loc[ suggestions.index ])
compare_to_seed(quaternaries.loc[ suggestions.index ], seed_df)

- beta**0.5:0:  0.3234480529009444
- beta**0.5:1:  0.32354201177926795
- beta**0.5:2:  0.32363537589505814
- beta**0.5:3:  0.3237281525776492


,Au%,Ag%,Cu%,Co%,Ni%,Pt%,Pd%,Sn%
6474,0.3,0.1,0.1,0.0,0.5,0.0,0.0,0.0
5449,0.2,0.1,0.1,0.0,0.6,0.0,0.0,0.0
6310,0.3,0.0,0.2,0.4,0.1,0.0,0.0,0.0
7076,0.4,0.1,0.1,0.0,0.4,0.0,0.0,0.0


           Au%  Ag%   Cu%   Co%   Ni%  Pt%  Pd%  Sn%  target
suggested  0.3  0.1  0.10  0.00  0.50  0.0  0.0  0.0     NaN
inseed     0.3  0.0  0.09  0.12  0.49  0.0  0.0  0.0     0.0
            Au%  Ag%   Cu%  Co%  Ni%  Pt%  Pd%  Sn%  target
suggested  0.20  0.1  0.10  0.0  0.6  0.0  0.0  0.0     NaN
inseed     0.23  0.0  0.17  0.0  0.6  0.0  0.0  0.0     0.0
            Au%  Ag%   Cu%   Co%  Ni%  Pt%  Pd%  Sn%  target
suggested  0.30  0.0  0.20  0.40  0.1  0.0  0.0  0.0     NaN
inseed     0.21  0.0  0.35  0.44  0.0  0.0  0.0  0.0     0.0
            Au%  Ag%   Cu%   Co%   Ni%  Pt%  Pd%  Sn%  target
suggested  0.40  0.1  0.10  0.00  0.40  0.0  0.0  0.0     NaN
inseed     0.47  0.0  0.06  0.11  0.36  0.0  0.0  0.0     0.0


In [8]:
new_raw_data = """
Ni%	Cu%	Ag%	Au%	Co%
45.71	7.11	7.81	39.37	0
38.42	6.87	11.52	43.19	0
37.13	6.14	13.34	43.39	0
37.61	6.33	14.41	41.65	0
41.49	6.51	8.42	43.58	0
38.63	6.72	6.61	48.04	0
40.04	5.18	13.91	40.87	0
40.46	4.98	14.55	40.01	0
40.36	6.04	7.95	45.64	0
37.9	5.35	15.51	41.24	0
41.92	5.58	9.75	42.76	0
9.35	14.33	0	33.11	43.21
10.1	15.14	0	31.31	43.45
10.92	14.98	0	30.16	43.95
10.63	14.72	0	31.63	43.01
"""
seed_df, seed_data, quaternaries, quaternary_feats = update_with_new_data(suggestions, new_raw_data, seed_df, seed_data, 
                                                                 quaternaries, quaternary_feats, round_number=round_number,
                                                                 elements=elements, measured=0)
round_number+=1

ElementProperty:   0%|          | 0/15 [00:00<?, ?it/s]

## Round 3
*Optimization agent's suggestions*:

In [9]:
agent = EmbedCompGPUCB(n_query=4)
suggestions = agent.get_hypotheses(candidate_data=quaternary_feats, seed_data=seed_data)
display(quaternaries.loc[ suggestions.index ])
compare_to_seed(quaternaries.loc[ suggestions.index ], seed_df)

- beta**0.5:0:  0.324786992238756
- beta**0.5:1:  0.32487274551428946
- beta**0.5:2:  0.32495800125823715
- beta**0.5:3:  0.32504276509385366


,Au%,Ag%,Cu%,Co%,Ni%,Pt%,Pd%,Sn%
6940,0.4,0.0,0.1,0.4,0.0,0.0,0.1,0.0
6434,0.3,0.1,0.0,0.2,0.4,0.0,0.0,0.0
6307,0.3,0.0,0.2,0.3,0.2,0.0,0.0,0.0
6301,0.3,0.0,0.2,0.2,0.3,0.0,0.0,0.0


            Au%  Ag%  Cu%   Co%  Ni%  Pt%  Pd%  Sn%  target
suggested  0.40  0.0  0.1  0.40  0.0  0.0  0.1  0.0     NaN
inseed     0.38  0.0  0.0  0.42  0.0  0.0  0.1  0.1    -2.0
            Au%  Ag%  Cu%   Co%   Ni%  Pt%  Pd%  Sn%  target
suggested  0.30  0.1  0.0  0.20  0.40  0.0  0.0  0.0     NaN
inseed     0.32  0.0  0.0  0.29  0.39  0.0  0.0  0.0     0.0
           Au%  Ag%   Cu%   Co%   Ni%  Pt%  Pd%  Sn%  target
suggested  0.3  0.0  0.20  0.30  0.20  0.0  0.0  0.0     NaN
inseed     0.3  0.0  0.15  0.44  0.11  0.0  0.0  0.0     0.0
            Au%  Ag%  Cu%   Co%   Ni%  Pt%  Pd%  Sn%  target
suggested  0.30  0.0  0.2  0.20  0.30  0.0  0.0  0.0     NaN
inseed     0.34  0.0  0.1  0.22  0.34  0.0  0.0  0.0     0.0


In [10]:
new_raw_data = """
Ni%	Co%	Ag%	Au%	Cu%	Pd%
55.4	29.9	5.8	7.5	1.5	0.0
55.5	29.6	4.5	7.8	2.6	0.0
56.2	29.6	4.2	6.3	3.7	0.0
63.1	30.2	2.8	3.9	0	0.0
63.8	30.2	2.3	3.7	0	0.0
62.9	30.2	1.4	3.5	2.1	0.0
18.8	39.7	0	20.9	20.6	0.0
22.8	40.4	0	28.2	8.5	0.0
20	42	0	19	19	0.0
24.4	24.6	0	35.7	15.3	0.0
22.9	24.5	0	43	9.6	0.0
25.4	26.8	0	28.8	19.1	0.0
25.3	26.1	0	25.3	16.2	0.0
0	55	0	24.6	13.2	7.3
0	55.7	0	24.1	13.5	6.7
0	53.4	0	24.8	14.3	7.4
0	56.4	0	22.7	13.7	7.2
"""
seed_df, seed_data, quaternaries, quaternary_feats = update_with_new_data(suggestions, new_raw_data, seed_df, seed_data, 
                                                                 quaternaries, quaternary_feats, round_number=round_number,
                                                                 elements=elements, measured=0)
round_number+=1

ElementProperty:   0%|          | 0/17 [00:00<?, ?it/s]

## Exploratory Rounds
### Pentanary SINP discovery

In [11]:
pentanaries = candidate_data[ ((candidate_data != 0).sum(axis=1) == 5)]
pentanary_feats = candidate_feats.loc[pentanaries.index]
agent = EmbedCompGPUCB(n_query=10)
suggestions_pentanaries = agent.get_hypotheses(candidate_data=pentanary_feats, seed_data=seed_data)
display(pentanaries.loc[ suggestions_pentanaries.index ])
compare_to_seed(pentanaries.loc[ suggestions_pentanaries.index ], seed_df)

- beta**0.5:0:  0.3255641307247349
- beta**0.5:1:  0.3256422790572916
- beta**0.5:2:  0.3257200130928286
- beta**0.5:3:  0.32579733711234043
- beta**0.5:4:  0.32587425533129716
- beta**0.5:5:  0.3259507719009695
- beta**0.5:6:  0.32602689090972103
- beta**0.5:7:  0.32610261638426785
- beta**0.5:8:  0.32617795229090707
- beta**0.5:9:  0.3262529025367149


,Au%,Ag%,Cu%,Co%,Ni%,Pt%,Pd%,Sn%
6250,0.3,0.0,0.1,0.4,0.1,0.0,0.1,0.0
6243,0.3,0.0,0.1,0.3,0.1,0.0,0.2,0.0
5073,0.2,0.0,0.1,0.2,0.4,0.0,0.1,0.0
5102,0.2,0.0,0.1,0.5,0.1,0.0,0.1,0.0
6306,0.3,0.0,0.2,0.3,0.1,0.0,0.1,0.0
5484,0.2,0.1,0.1,0.2,0.4,0.0,0.0,0.0
6221,0.3,0.0,0.1,0.1,0.4,0.0,0.1,0.0
6489,0.3,0.1,0.1,0.1,0.4,0.0,0.0,0.0
5182,0.2,0.0,0.2,0.4,0.1,0.0,0.1,0.0
5470,0.2,0.1,0.1,0.1,0.5,0.0,0.0,0.0


            Au%  Ag%   Cu%   Co%  Ni%  Pt%  Pd%  Sn%  target
suggested  0.30  0.0  0.10  0.40  0.1  0.0  0.1  0.0     NaN
inseed     0.31  0.0  0.15  0.43  0.1  0.0  0.0  0.0     0.0
            Au%  Ag%   Cu%   Co%  Ni%  Pt%  Pd%  Sn%  target
suggested  0.30  0.0  0.10  0.30  0.1  0.0  0.2  0.0     NaN
inseed     0.31  0.0  0.15  0.43  0.1  0.0  0.0  0.0     0.0
            Au%  Ag%   Cu%   Co%   Ni%  Pt%  Pd%  Sn%  target
suggested  0.20  0.0  0.10  0.20  0.40  0.0  0.1  0.0     NaN
inseed     0.21  0.0  0.22  0.14  0.43  0.0  0.0  0.0     0.0
            Au%  Ag%   Cu%   Co%  Ni%  Pt%   Pd%  Sn%  target
suggested  0.20  0.0  0.10  0.50  0.1  0.0  0.10  0.0     NaN
inseed     0.25  0.0  0.14  0.53  0.0  0.0  0.07  0.0     0.0
            Au%  Ag%   Cu%   Co%  Ni%  Pt%  Pd%  Sn%  target
suggested  0.30  0.0  0.20  0.30  0.1  0.0  0.1  0.0     NaN
inseed     0.31  0.0  0.15  0.43  0.1  0.0  0.0  0.0     0.0
            Au%  Ag%   Cu%   Co%   Ni%  Pt%  Pd%  Sn%  target
suggested  0.20  

In [12]:
new_raw_data = """
Co%	Ni%	Cu%	Pd%	Ag%	Au%
32.9	10.6	7.3	12.2	0	37
29.5	9.5	6.4	20.5	0	34.2
33.9	10	7	12.3	0	36.8
32.8	9.9	7	13.6	0	36.7
43.9	18	14.7	9	0	14.4
46.3	18.4	13.5	8.2	0	13.6
44.8	18	14.1	8.8	0	14.2
19.4	39.8	10.4	11	0	19.4
19.5	40.2	10.3	10.8	0	19.3
19.7	40	10.4	10.5	0	19.3
19	40.1	10.2	11	0	19.7
22.9	45.1	7.8	0	3.5	20.6
23.1	44.8	7	0	5.9	19.1
23.5	45	7.3	0	5	19.3
22.8	44	7.2	0	6.6	19.5
8.2	23.5	6.5	0	6.3	55.5
7.6	22.6	6.1	0	9.9	53.8
7.9	24.1	6.2	0	7.8	54
7.8	23.2	6	0	10	53
"""

suggestions_targeted_by_team = [6250,6243,5073,5484,6489]

seed_df, seed_data, pentanaries, pentanary_feats = update_with_new_data(suggestions_pentanaries.loc[suggestions_targeted_by_team], 
                                                                        new_raw_data, seed_df, seed_data, 
                                                                         pentanaries, pentanary_feats, 
                                                                         round_number=round_number,
                                                                         elements=elements, measured=0)
round_number+=1

ElementProperty:   0%|          | 0/19 [00:00<?, ?it/s]

### Hexanary SINP discovery

In [13]:
hexanaries = candidate_data[ ((candidate_data != 0).sum(axis=1) == 6)]
hexanaries_feats = candidate_feats.loc[hexanaries.index]
agent = EmbedCompGPUCB(n_query=10)
suggestions_hexanaries = agent.get_hypotheses(candidate_data=hexanaries_feats, seed_data=seed_data)
display(hexanaries.loc[ suggestions_hexanaries.index ].head(10))
compare_to_seed(hexanaries.loc[ suggestions_hexanaries.index ], seed_df)

- beta**0.5:0:  0.3185879466972021
- beta**0.5:1:  0.3186607656682954
- beta**0.5:2:  0.3187332317704999
- beta**0.5:3:  0.31880534833555335
- beta**0.5:4:  0.31887711864859103
- beta**0.5:5:  0.3189485459490076
- beta**0.5:6:  0.31901963343129863
- beta**0.5:7:  0.31909038424588326
- beta**0.5:8:  0.3191608014999082
- beta**0.5:9:  0.3192308882580335


,Au%,Ag%,Cu%,Co%,Ni%,Pt%,Pd%,Sn%
3837,0.1,0.1,0.1,0.2,0.4,0.0,0.1,0.0
5498,0.2,0.1,0.1,0.4,0.1,0.0,0.1,0.0
6504,0.3,0.1,0.1,0.3,0.1,0.0,0.1,0.0
3866,0.1,0.1,0.1,0.5,0.1,0.0,0.1,0.0
5469,0.2,0.1,0.1,0.1,0.4,0.0,0.1,0.0
3816,0.1,0.1,0.1,0.1,0.5,0.0,0.1,0.0
6488,0.3,0.1,0.1,0.1,0.3,0.0,0.1,0.0
3835,0.1,0.1,0.1,0.2,0.3,0.0,0.2,0.0
5479,0.2,0.1,0.1,0.2,0.1,0.0,0.3,0.0
3859,0.1,0.1,0.1,0.4,0.1,0.0,0.2,0.0


            Au%  Ag%  Cu%  Co%  Ni%  Pt%  Pd%  Sn%  target
suggested  0.10  0.1  0.1  0.2  0.4  0.0  0.1  0.0     NaN
inseed     0.19  0.0  0.1  0.2  0.4  0.0  0.1  0.0     0.0
            Au%  Ag%   Cu%   Co%   Ni%  Pt%   Pd%  Sn%  target
suggested  0.20  0.1  0.10  0.40  0.10  0.0  0.10  0.0     NaN
inseed     0.14  0.0  0.15  0.44  0.18  0.0  0.09  0.0     0.0
            Au%  Ag%   Cu%   Co%   Ni%  Pt%   Pd%  Sn%  target
suggested  0.30  0.1  0.10  0.30  0.10  0.0  0.10  0.0     NaN
inseed     0.37  0.0  0.07  0.33  0.11  0.0  0.12  0.0     0.0
            Au%  Ag%   Cu%   Co%   Ni%  Pt%   Pd%  Sn%  target
suggested  0.10  0.1  0.10  0.50  0.10  0.0  0.10  0.0     NaN
inseed     0.14  0.0  0.14  0.46  0.18  0.0  0.08  0.0     0.0
           Au%  Ag%  Cu%   Co%  Ni%  Pt%   Pd%  Sn%  target
suggested  0.2  0.1  0.1  0.10  0.4  0.0  0.10  0.0     NaN
inseed     0.2  0.0  0.1  0.19  0.4  0.0  0.11  0.0     0.0
            Au%  Ag%  Cu%   Co%  Ni%  Pt%   Pd%  Sn%  target
suggested  0.10